In [7]:
from datetime import datetime, timedelta

import numpy as np
import pandas as pd

def input(filename):
    with open(filename, 'r') as input:
        data = [x.strip() for x in input]

    return data

records = input('day4.txt')
records.sort()
history = pd.DataFrame(columns=['day', 'guard'])
minutes = pd.DataFrame(np.zeros((1, 60), dtype=int), columns=np.arange(60))

In [8]:
records = [
        '[1518-11-01 00:00] Guard #10 begins shift',
        '[1518-11-01 00:05] falls asleep',
        '[1518-11-01 00:25] wakes up',
        '[1518-11-01 00:30] falls asleep',
        '[1518-11-01 00:55] wakes up',
        '[1518-11-01 23:58] Guard #99 begins shift',
        '[1518-11-02 00:40] falls asleep',
        '[1518-11-02 00:50] wakes up',
        '[1518-11-03 00:05] Guard #10 begins shift',
        '[1518-11-03 00:24] falls asleep',
        '[1518-11-03 00:29] wakes up',
        '[1518-11-04 00:02] Guard #99 begins shift',
        '[1518-11-04 00:36] falls asleep',
        '[1518-11-04 00:46] wakes up',
        '[1518-11-05 00:03] Guard #99 begins shift',
        '[1518-11-05 00:45] falls asleep',
        '[1518-11-05 00:55] wakes up'
    ]

In [9]:
for r in records:
        s = r.split()
        ts = datetime.strptime(' '.join([s[0], s[1]]).strip('[]'), '%Y-%m-%d %H:%M') + timedelta(days=36500 * 5)
        day = pd.Timestamp(ts).round('D').date()
        if 'Guard' in r:
            result = pd.concat([pd.DataFrame([{'day': day, 'ts': ts, 'guard': s[3]}]), minutes], axis=1)
            history = pd.concat([history, result], sort=True)
        if 'falls asleep' in r:
            history.loc[history['day'] == day, list(range(ts.minute, 60))] = 1
        if 'wakes up' in r:
            history.loc[history['day'] == day, list(range(ts.minute, 60))] = 0

In [10]:
history['total'] = history[np.arange(60)].sum(axis=1)

In [11]:
summary = history[['day', 'guard', 'total']]
by_guards = history[['guard', 'total']].groupby('guard').sum()
g = by_guards.idxmax()[0]

In [12]:
summary

,day,guard,total
0,2018-07-02,#10,45.0
0,2018-07-03,#99,10.0
0,2018-07-04,#10,5.0
0,2018-07-05,#99,10.0
0,2018-07-06,#99,10.0


In [13]:
guard_minutes = history.loc[history['guard'] == g, np.arange(60)].sum()
m = guard_minutes.idxmax()

In [14]:
print('Part 1\nguard: ' + g + ', minute: ' + str(m))

Part 1
guard: #10, minute: 24


In [15]:
by_minutes = history[['guard', *np.arange(60)]].groupby('guard').sum()
top_minute = by_minutes.max().idxmax()
top_guard = by_minutes[[top_minute]].idxmax().item()

In [16]:
print('Part 2\nguard: ' + top_guard + ', minute: ' + str(top_minute))

Part 2
guard: #99, minute: 45
